In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 75%; }
</style>
"""))

In [2]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
from src.learning.models import create_cnn_nvidia, create_cnn_alone, create_cnn_alone_categorical, \
                                create_mlp, create_multi_model_2, create_multi_model_3, \
                                create_cnn_nvidia_deep, create_cnn_nvidia_shallow, create_cnn_nvidia_middle, create_multi_cnn

In [3]:
def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Epoch')
    x = np.arange(0, len(plot_elems[0]['data']), 1)
    
    for plot_elem in plot_elems:
        plt.errorbar(x, plot_elem['data'], yerr=plot_elem['error'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o', capsize=5)

    plt.grid(axis='x')
    plt.legend(loc='best', prop={'size': 15})
    plt.show()
    plt.savefig('./' + title + '.png')
    
def get_model_num(model_path, model_prefix):
    model_files = [fn for fn in os.listdir(model_path) if fn.startswith(model_prefix) and fn.endswith('.h5')]
    # expected format is "model_n1_m1_9.h5"
    existing_nums = [int(fn.split('_')[3].split('.')[0]) for fn in model_files]
    
    latest_num = sorted(existing_nums)[-1]
    return int(latest_num) + 1

In [4]:
config_manager = ConfigurationManager()
config = config_manager.config

memory_variants = [(1, 1), (4, 1), (8, 1)]
memory_variants = memory_variants[0:2]
memory = memory_variants[0]

model_path = '../../training/models/'

In [6]:
# Model experiments

epochs = 6
batch_size = 32
verbose = 1

losses = []
val_losses = []

generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer')
frame_shape, numeric_shape, diff_shape = generator.get_shapes()
tqdm.write('Target shape: {}'.format(diff_shape))
tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))

models = []

#mlp = create_mlp(input_shape=numeric_shape)
#cnn = create_cnn_nvidia(input_shape=frame_shape)
#models.append((create_multi_model_3(mlp, cnn, output_shape=diff_shape), generator.generate_with_numeric))

#cnn_1 = create_cnn_nvidia_shallow(input_shape=frame_shape)
cnn_2 = create_cnn_nvidia_deep(input_shape=frame_shape)
cnn_3 = create_cnn_nvidia_middle(input_shape=frame_shape)
models.append((create_multi_cnn([cnn_2, cnn_3], output_shape=diff_shape), generator.generate_with_super_duper))

for model, generate_method in tqdm(models):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)

    hist = model.fit(generate_method(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generate_method(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

os.system("printf '\a'")

Target shape: 2
Input shapes: (50, 180, 3); (2,)


n1_m1
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 3709 steps, validate for 927 steps
Epoch 1/6
3709/3709 [==============================] - 273s 74ms/step - loss: 0.5821 - val_loss: 0.2551
Epoch 2/6
3709/3709 [==============================] - 272s 73ms/step - loss: 0.2364 - val_loss: 0.2175
Epoch 3/6
3709/3709 [==============================] - 273s 74ms/step - loss: 0.2167 - val_loss: 0.2060
Epoch 4/6
3709/3709 [==============================] - 273s 74ms/step - loss: 0.2106 - val_loss: 0.1975
Epoch 5/6
3709/3709 [==============================] - 273s 74ms/step - loss: 0.2071 - val_loss: 0.1953
Epoch 6/6
3709/3709 [==============================] - 274s 74ms/step - loss: 0.2053 - val_loss: 0.1938
Loss per epoch: [0.582085972097917, 0.23644311708014235, 0.21672129048896818, 0.21056675145602927, 0.20709500373631162, 0.20533003760303375]
Validation loss per epoch: [0.25513000749332876, 0.217510798039817, 0.20599048775183731, 0.19752811286055927, 0.19530175600558553, 0.1

0

In [ ]:
# Memory experiments on steering + throttle models

epochs = 8
batch_size = 32
verbose = 1

losses = []
val_losses = []
    
for memory in tqdm(memory_variants):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)
    
    generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer')
    frame_shape, numeric_shape, diff_shape = generator.get_shapes()
    tqdm.write('Target shape: {}'.format(diff_shape))
    tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))
    
    #model = create_cnn_alone(input_shape=frame_shape, output_shape=diff_shape)
    
    mlp = create_mlp(input_shape=numeric_shape)
    cnn = create_cnn_nvidia(input_shape=frame_shape)
    model = create_multi_model_3(mlp, cnn, output_shape=diff_shape)

    hist = model.fit(generator.generate_with_numeric(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generator.generate_with_numeric(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()
    
os.system("printf '\a'")

In [ ]:
# Memory experiments on gear models

epochs = 5
batch_size = 32
verbose = 1

gear_losses = []
gear_val_losses = []
    
for memory in tqdm(memory_variants):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)
    
    generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='all')
    frame_shape, numeric_shape, diff_shape = generator.get_shapes()
    tqdm.write('Target shape: {}'.format(diff_shape))
    tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))
    
    mlp = create_mlp(input_shape=numeric_shape)
    cnn = create_cnn_nvidia(input_shape=frame_shape)
    gear_model = create_multi_model_2(mlp, cnn, output_shape=2)

    hist = gear_model.fit(generator.generate_with_numeric(data='train'),
                          steps_per_epoch=generator.train_batch_count,
                          validation_data=generator.generate_with_numeric(data='test'),
                          validation_steps=generator.test_batch_count,
                          epochs=epochs, verbose=verbose)

    model_file_prefix = 'gear_model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(gear_model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    gear_model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    gear_losses.append(current_loss)
    gear_val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

In [ ]:
loss_data = []
val_loss_data = []

for i in range(0, len(losses)):
    val_loss_data.append({'data': val_losses[i], 'label': key, 'alpha': 1.0})
    loss_data.append({'data': losses[i], 'label': key, 'alpha': 1.0})
        
plot_stuff("val losses", val_loss_data, figsize=(10, 14))
plot_stuff("losses", loss_data, figsize=(10, 14))

In [ ]:
import tensorflow as tf

generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=32, column_mode='steer', separate_files=True)
frame_shape, numeric_shape, diff_shape = generator.get_shapes()

shapes = ([50, 180, 12], ())
types = (np.float32, np.float32)

train_dataset = tf.data.Dataset.from_generator(generator.generate_single_train, output_types=types, output_shapes=shapes)
train_dataset = train_dataset.batch(batch_size=32)
train_dataset = train_dataset.shuffle(buffer_size=10)

val_dataset = tf.data.Dataset.from_generator(generator.generate_single_test, output_types=types, output_shapes=shapes)
val_dataset = val_dataset.batch(batch_size=32)
val_dataset = val_dataset.shuffle(buffer_size=10)

model = create_cnn_alone(input_shape=frame_shape, output_shape=diff_shape)

hist = model.fit(train_dataset,
                 steps_per_epoch=generator.train_batch_count,
                 validation_data=val_dataset,
                 validation_steps=generator.test_batch_count,
                 epochs=10, verbose=1)